In [1]:
from bs4 import BeautifulSoup
import urllib
import json
from tqdm import tqdm

from urllib.request import urlretrieve

import requests

In [2]:
latest_post_id = 13967
target_number = 1000 # 얼마나 크롤링할지
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55'}

titles = []

In [3]:
for i in tqdm(range(latest_post_id, latest_post_id - target_number, -1)):
    style_id = f'musinsa_codimap_style_{i}'

    url = f'https://store.musinsa.com/app/codimap/views/{i}?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1'
    req = requests.get(url, headers=headers)
    req.raise_for_status()
    soup = BeautifulSoup(req.text, 'lxml')
    contents = soup.find('div', attrs={'class': 'styling_content codimap-contents'})

    # Style 요소들
    style_title = contents.find('h2').text  # 이름
    style_link = url
    style_date = contents.find('p', attrs={'class': 'styling_date'}).text[:10]  # 게시날짜
    style_description = contents.find('p', attrs={'class': 'styling_txt'}).text  # 스타일 한줄 설명
    style_tags = contents.find_all('div', attrs={'class': 'ui-tag-list'}) # 해시 태그들
    style_tag = [tag.text[1:] for tag in style_tags[0]]
    style_tag = list(filter(None, style_tag))
    style_img_url = 'https:' + contents.find('div', attrs={'class': 'codimap-cont'}).find('img')['src'][:-13]  # 스타일 디피 사진

    urllib.request.urlretrieve(style_img_url, f'raw/img/img_style/{style_id}.jpg')
    # 저장
    style_info = dict()
    style_info['title'] = style_title
    style_info['style_id'] = style_id
    style_info['link'] = style_link
    style_info['date'] = style_date
    style_info['description'] = style_description
    style_info['tags'] = style_tag
    with open(f"raw/info/info_style/{style_id}.json", 'w', encoding='utf-8') as file1:
        json.dump(style_info, file1, indent="\t", ensure_ascii=False)
    file1.close()

    # Style에 속하는 아이템들
    item_section = contents.find('div', attrs={'class': 'styling_list swiper-wrapper'})
    items = item_section.find_all('div')

    for j, item in enumerate(items):
        if item.attrs['class'][0] in ['box-img', 'price']: 
            continue
        item_id = f'{style_id}_item_{j}'        
        # 아이템 요소들
        item_title = item.find('a', attrs={'class': 'brand_item'}).text        
        item_link = 'https://store.musinsa.com/' + item.find('a', attrs={'class': 'styling_img'})['href']
        item_img_url ='https:'+ item.find('img')['src']
        urllib.request.urlretrieve(item_img_url, f'raw/img/img_item/{item_id}.jpg')

        item_info = dict()
        item_info['title'] = item_title
        item_info['style_id'] = style_id
        item_info['item_id'] = item_id
        item_info['link'] = item_link
        item_info['title'] = item_title
        with open(f"raw/info/info_item/{item_id}.json", 'w', encoding='utf-8') as file2:
            json.dump(item_info, file2, indent="\t", ensure_ascii=False)
        file2.close()

 77%|█████████████████████████████████████████████████████████████▏                 | 774/1000 [08:14<02:24,  1.57it/s]


AttributeError: 'NoneType' object has no attribute 'find'